0.0 Imports and setup

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import tensorflow_datasets as tfds

import montecarlo_dropout


2023-12-16 09:46:41.395291: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-16 09:46:41.761755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 09:46:41.761912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 09:46:41.826907: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 09:46:41.963891: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-16 09:46:41.965354: I tensorflow/core/platform/cpu_feature_guard.cc:1

Dataset cifar for validation crossplatform (not needed at the moment but maybe helpfull later)

In [12]:
(ds_test), ds_info = tfds.load(
    'imagenet_v2',
    split=['test[:90%]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /home/emilhillebrand/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0.incompleteSBAT6E/imagen…

Dataset imagenet_v2 downloaded and prepared to /home/emilhillebrand/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0. Subsequent calls will reuse this data.


2023-12-14 16:55:47.971231: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Load Image

In [3]:
testtabby = load_img("./Pictures/tabby-cat.jpeg")
testtabby = testtabby.resize((299, 299))
#ytest = img_to_array(y_test)
testtabby = img_to_array(testtabby)
testtabby = np.expand_dims(testtabby, axis=0)  

Normalize function (not needed at the moment)

In [2]:


def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(299)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)



NameError: name 'ds_test' is not defined

Model no Monte Carlo

In [5]:
modelNoMC = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

modelNoMC.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]

ModelMC Test extern

In [ ]:
model = tf.keras.applications.MobileNetV2()

model.save("./model.keras")

modelMC = montecarlo_dropout.MontecarloDropout()
modelMC.build_model("./model.keras")

#modelMC = build_model(modelNoMC)
modelMC.summary()

Make a MC D model

In [7]:

layers = []
dropout_rate = 0.4
#model = tf.keras.models.Sequential()
#for i in range(len(modelNoMC.layers)):
#    model.add(modelNoMC.layers.pop())
#    model.add(tf.keras.layers.Dropout(dropout_rate))
    #print(i)
#    layers.append(modelNoMC.layers[i])
    
#print((layers))


#layers.reverse

input_layer = modelNoMC.layers[0]
x = modelNoMC.layers[1](input_layer)

for i in range(len(modelNoMC.layers)-2):
    x = modelNoMC.layers[i+2](x)
    x = tf.keras.layers.Dropout(dropout_rate, training=True)(x)
    if(i == (len(layers)-2)):
        output_layer = modelNoMC.layers[i+2](x)

modelMC = keras.model.Model(inputs=input_layer, outputs=output_layer)

#modelMC.build((None, 299,299, 3))
#predictions = modelMC.predict(preprocess)
modelMC.summary()

TypeError: Inputs to a layer should be tensors. Got '<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f067a1d3010>' (of type <class 'keras.src.layers.convolutional.conv2d.Conv2D'>) as input for layer 'block1_conv1_bn'.

Weight adjustment due to dropout

In [121]:
WEIGHTS = modelNoMC.get_weights()
WEIGHTS[0] *= 1/0.4
modelNoMC.set_weights(WEIGHTS)

evaluate Model

In [13]:
preprocess = keras.applications.xception.preprocess_input(testtabby[None])

model Compile

In [6]:
modelNoMC.compile(
optimizer='Adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

predict

In [34]:
predictions = modelNoMC.predict(testtabby)
predictions = keras.applications.xception.decode_predictions(predictions)
print(predictions)

1/1 [==============================] - 0s 366ms/step
[[('n02123045', 'tabby', 0.6505307), ('n02123159', 'tiger_cat', 0.23030522), ('n02124075', 'Egyptian_cat', 0.01496593), ('n02127052', 'lynx', 0.0042105187), ('n02123394', 'Persian_cat', 0.0010178831)]]


MonteCarlo predict

In [33]:
y_samples = []
for i in range(10):
    predictions = modelNoMC.predict(testtabby)
    y_samples.append(keras.applications.xception.decode_predictions(predictions))
    #print(y_samples[i])

1/1 [==============================] - 0s 382ms/step


Output MC D

In [32]:
counter = 0
for i in range(len(y_samples)):
    #print(y_samples[i][0][0][1])
    if(y_samples[i][0][0][1] == "tabby"):
        counter = counter +1
y_samples_mean = counter / len(y_samples)
print(y_samples_mean)

tabby
tabby
tabby
tabby
tabby
tabby
tabby
tabby
tabby
tabby
1.0
